In [32]:
from pathlib import Path

import torch

## Dataset

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-04-27 09:43:47--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.01s   

2024-04-27 09:43:47 (83.1 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [19]:
dataset_path = Path.cwd() / "input.txt"

In [21]:
text = dataset_path.read_text()

In [22]:
print(f"length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [24]:
print(text[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


## Tokenize

In [27]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [136]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

In [137]:
print(encode("Starting gpt"))
print(decode(encode("Starting gpt")))

[31, 58, 39, 56, 58, 47, 52, 45, 1, 45, 54, 58]
Starting gpt


In [42]:
## Encode entire text
data = torch.tensor(encode(text), dtype=torch.long)

In [40]:
data.shape, data.dtype

(torch.Size([1115394]), torch.int64)

In [41]:
data[:30]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43])

## Split dataset

In [43]:
n = int(.9 * len(data))

In [44]:
train_data = data[:n]
val_data = data[n:]

In [45]:
train_data.shape, val_data.shape

(torch.Size([1003854]), torch.Size([111540]))

## Chunk Dataset

In [48]:
block_size = 8
train_data[: block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [51]:
X = train_data[:block_size]
y = train_data[1:block_size + 1]

In [56]:
for t in range(block_size):
    context = X[:t + 1]
    target = y[t]
    print(f"{context=} {target=}")

context=tensor([18]) target=tensor(47)
context=tensor([18, 47]) target=tensor(56)
context=tensor([18, 47, 56]) target=tensor(57)
context=tensor([18, 47, 56, 57]) target=tensor(58)
context=tensor([18, 47, 56, 57, 58]) target=tensor(1)
context=tensor([18, 47, 56, 57, 58,  1]) target=tensor(15)
context=tensor([18, 47, 56, 57, 58,  1, 15]) target=tensor(47)
context=tensor([18, 47, 56, 57, 58,  1, 15, 47]) target=tensor(58)


In [57]:
torch.manual_seed(1337)

batch_size = 4
block_size = 8

In [64]:
torch.randint(len(train_data) - block_size, (batch_size, ))

tensor([ 76049, 234249, 934904, 560986])

In [76]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, )) 
    x = torch.stack([data[i: i + block_size] for i in ix])
    y = torch.stack([data[i+1 : i + block_size + 1] for i in ix]   )
    return x, y

In [84]:
xb, yb = get_batch('train')

In [85]:
xb.shape, yb.shape

(torch.Size([4, 8]), torch.Size([4, 8]))

In [86]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"{context=} {target=}")

    break

context=tensor([43]) target=tensor(43)
context=tensor([43, 43]) target=tensor(51)
context=tensor([43, 43, 51]) target=tensor(5)
context=tensor([43, 43, 51,  5]) target=tensor(42)
context=tensor([43, 43, 51,  5, 42]) target=tensor(1)
context=tensor([43, 43, 51,  5, 42,  1]) target=tensor(40)
context=tensor([43, 43, 51,  5, 42,  1, 40]) target=tensor(59)
context=tensor([43, 43, 51,  5, 42,  1, 40, 59]) target=tensor(56)


## Bigram LM

In [87]:
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

In [156]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size: int):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets = None):
        logits = self.token_embedding(idx)
        if targets is None: 
            loss = None 
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_token):
        for _ in range(max_new_token):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [157]:
m = BigramLanguageModel(vocab_size)

In [158]:
idx.shape, xb.shape

(torch.Size([1, 1]), torch.Size([4, 8]))

In [159]:
logits, loss = m(idx, )
logits.shape

torch.Size([1, 1, 65])

In [160]:
idx = torch.zeros((1, 1), dtype=torch.long)
max_token = 100


In [161]:
print(decode(m.generate(idx, max_token)[0].tolist()))


.zNtcVC$'NW
'Kl-g:uV&&bcMIhdHm EJQ!zmhKPYP'Qy JlJKFrhV3S'vOisaKsaeB'?Y
KP'AvN 'QHHQJbldzOcILsAO;n'YK


## 1: Train Model

In [169]:
from tqdm import tqdm, trange

In [164]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [175]:
@torch.no_grad
def estimate_loss(model: nn.Module, eval_iters: int):
    out = {}
    model.eval()

    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, y = get_batch(split)
            logits, loss = model(X, y)
            losses[k] = loss.item()
        
        out[split] = losses.mean()
    model.train()
    return out

In [181]:
batch_size = 32
max_iter = 200
eval_interval = 10

for iter in range(max_iter):
    if iter % eval_interval == 0:
        losses = estimate_loss(m, max_iter)
        print(f"{iter=} train_loss: `{losses['train']:.4f}` val_loss `{losses['val']:.4f}`")


    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()


iter=0 train_loss: `2.7179` val_loss `2.7287`
iter=10 train_loss: `2.7134` val_loss `2.7231`
iter=20 train_loss: `2.7164` val_loss `2.7262`


iter=30 train_loss: `2.7131` val_loss `2.7251`
iter=40 train_loss: `2.7123` val_loss `2.7231`
iter=50 train_loss: `2.7102` val_loss `2.7185`
iter=60 train_loss: `2.7081` val_loss `2.7126`
iter=70 train_loss: `2.7088` val_loss `2.7155`
iter=80 train_loss: `2.6987` val_loss `2.7211`
iter=90 train_loss: `2.7027` val_loss `2.7159`
iter=100 train_loss: `2.7080` val_loss `2.7145`
iter=110 train_loss: `2.7015` val_loss `2.7156`
iter=120 train_loss: `2.7005` val_loss `2.6958`
iter=130 train_loss: `2.6965` val_loss `2.7098`
iter=140 train_loss: `2.6956` val_loss `2.6979`
iter=150 train_loss: `2.6929` val_loss `2.7084`
iter=160 train_loss: `2.6917` val_loss `2.6998`
iter=170 train_loss: `2.6827` val_loss `2.7030`
iter=180 train_loss: `2.6854` val_loss `2.6947`
iter=190 train_loss: `2.6911` val_loss `2.6977`


In [182]:
idx = torch.zeros((1, 1), dtype=torch.long)
max_token = 500

print(decode(m.generate(idx, max_token)[0].tolist()))


Ot&

&d sea CovGurededYI'watstRCotur, nd ny G&Tidif vNoDOu,OUTi$anerh, mewF:,Lageyeklen yoWvfiler 
G bo, bum.;IG oUP.
I herlind thul-IOFeno pr?hevcupGghoo,
ciwhesimxin Zpl fOqbll.
QusIZGh,UEaWh,
RiFTL:knF:


BWe,
A$k .
LDaruly otal:J3Pgh herechot ir, theKZWhwhendwarERO akeUTY INTCHERif,TVis, IUCJUpthovyofoly$Q&;Qfo ais:RCEmeanshJUG d, fvirf
SANDave OPJbemhemf we

HOf ANU!
ALon, putBMwB'enths n CANAG ldl Z-z ICY,
ELYKI u m y papeadDa abhe.Tf an, m;d
Afrare,K-usxisurd g;
HES:

kvof;wd dy dom l?
Wi


## The mathematical trick in self-attention

In [184]:
torch.tril(torch.ones(5, 5))

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])